# Project 1: Population & Food Supply

In [41]:
# %pip install wbdata
# %pip install pandas
# %pip install plotly

import wbdata
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
from datetime import datetime
import numpy as np

### Population Statistics Function

In [48]:
countries = wbdata.get_countries()
country_dict = {}

# Iterate over the WBSearchResult object, Create Dictionary.
# This dictionary is necessary so that we know which country
# names map to which 3-letter country codes.
for country in countries:
    country_code = country['id']
    country_name = country['name']

    # Add to the dictionary the country name.
    country_dict[country_name] = country_code

def int_to_str(num):
    """Convert the integer to the proper format."""
    if 0 <= num < 10:
        # Add a '0' prefix if it is a single digit.
        return f"0{num}"
    else:
        # Convert to string directly if it is a two-digit number.
        return str(num)

def population_range(year, sex, age_range, place):
    """This function will return the population for a certain age range."""
    sex_codes = {"people": "", "females": "FE", "males": "MA"}
    sex_used = sex_codes[sex]

    # Getting the lower and upper bounds for date into correct format.
    lower, upper = int_to_str(age_range[0]), int_to_str(age_range[1])
    range_string = lower + upper

    # Getting the country code via the name.
    country_code = country_dict.get(place)

    df = wbdata.get_dataframe({"SP.POP." + range_string + "." + sex_used: "Population"},
                              country= {country_code: place}).squeeze()
    df = df.to_frame().reset_index()
    population_total = int(df[df["date"] == str(year)]["Population"])
    return population_total

def over_80_pop(year, sex, place):
    sex_codes = {"people": "", "females": "FE", "males": "MA"}
    sex_used = sex_codes[sex]
    country_code = country_dict.get(place)

    df = wbdata.get_dataframe({"SP.POP." + "80UP" + "." + sex_used: "Population"},
                              country= {country_code: place}).squeeze()
    df = df.to_frame().reset_index()
    population_total = int(df[df["date"] == str(year)]["Population"])
    return population_total


def dict_helper(year, sex, age_range, place):
    """This will expand our function to include every age specified possible."""
    if len(age_range) == 1:
        age_range = [age_range[0], age_range[0]]
    elif age_range[1] < age_range[0]:
      raise ValueError(f"Please ensure that the second value in the range is greater than the first.")

    minimum_age, maximum_age = age_range
    possible_minimums = [i for i in range(0, 76, 5)]
    possible_maximums = [i for i in range(4, 80, 5)]

    my_dict = {}
    for age in range(minimum_age, maximum_age + 1):
        """Find the index in the possible ranges that includes the current age."""
        range_index = next((i for i, min_val in enumerate(possible_minimums) if
                            min_val <= age and age <= possible_maximums[i]), None)
        if range_index is not None:
            popl_value = population_range(year, sex, [possible_minimums[range_index], possible_maximums[range_index]], place) // 5
            my_dict[age] = popl_value
        elif age >= 80 and age <= 100:
            my_dict[age] = over_80_pop(year, sex, place) / 20
        else:
            raise ValueError(f"No age range available for age {age}")

    return my_dict


def population(year, sex, age_range, place):
    """This function ties everything together, returning population for given age ranges.
    During usage, please utilize the following format:
    Arguments to Use:
    Year (int): the specified year, works from 1960-2021.
    Sex (string): Anything from all, people, p, P, People, All, Everyone, female, females, f, Female, 
                Females, F, FE, male, males, m, Male, Males, M, MA works.
    Age Range (list with length 2, 2 integers): A list of the age bounds.
    Place (string): A string of the specified location.
    Example Usage: population(1975, "People", [5, 7], "Rwanda")
    """
    if place not in country_dict:
        valid_regions = ", ".join(country_dict.keys())
        raise ValueError(f"The region '{place}' is not valid. Please choose from the following regions: {valid_regions}")
    if sex in ["all", "people", "p", "P", "People", "All", "Everyone"]:
      female_dict = dict_helper(year, "females", age_range, place)
      male_dict = dict_helper(year, "males", age_range, place)
      return sum(female_dict.values()) + sum(male_dict.values())
    elif sex in ["female", "females", "f", "Female", "Females", "F", "FE"]:
      female_dict = dict_helper(year, "females", age_range, place)
      return sum(female_dict.values())
    elif sex in ["male", "males", "m", "Male", "Males", "M", "MA"]:
      male_dict = dict_helper(year, "males", age_range, place)
      return sum(male_dict.values())
        
population(1975, "People", [5, 7], "Haiti")

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling in

405675

In [49]:
population(2022, "people", [0,100], "World")

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling in

7997756448.5

### Popualtion Dataframe

In [50]:
def create_population_dataframe(regions, years, age_range):
    """Creates the population DataFrame based on the regions, years, and age ranges wanted.
    During usage, please utilize the following format:
    Arguments to Use:
    regions (list of strings, any length): the specified year, works from 1960-2021.
    years (list of ints, any length): Anything from all, people, p, P, People, All, Everyone, female, females, f, Female, 
                Females, F, FE, male, males, m, Male, Males, M, MA works.
    age_range (list of two ints): the age bounds specified, second must be greater than the first.
    """
    data = []

    # Check if age_range is a single age or a range.
    if len(age_range) == 1:
        full_age_range = [age_range[0]]
    else:
        full_age_range = list(range(age_range[0], age_range[1] + 1))

    # Iterate over each region, year, and age."
    for region in regions:
        for year in years:
            row = {'Region': region, 'Year': year}
            for age in full_age_range:
                male_population = population(year, 'male', [age], region)
                female_population = population(year, 'female', [age], region)
                total_population = male_population + female_population

                row[f'Male Population Age {age}'] = male_population
                row[f'Female Population Age {age}'] = female_population
                row[f'Total Population Age {age}'] = total_population

            data.append(row)

    # Create a DataFrame.
    df = pd.DataFrame(data)

    # Set the index.
    df.set_index(['Region', 'Year'], inplace=True)

    return df

In [51]:
population_df = create_population_dataframe(["Haiti"], [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000], [5, 70])
population_df

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/y6/q4h46fhn20scql1wyt3t4k600000gn/T/ipykernel_16393/152533637.py:38: FutureWarning:

Calling in

Male Population Age 5  Female Population Age 5  \
Region Year                                                   
Haiti  1990                  94025                    93617   
       1991                  96229                    95802   
       1992                  98512                    98066   
       1993                 100878                   100412   
       1994                 103264                   102775   
       1995                 105598                   105083   
       1996                 107755                   107209   
       1997                 109577                   108995   
       1998                 111005                   110381   
       1999                 112085                   111410   
       2000                 112886                   112158   

             Total Population Age 5  Male Population Age 6  \
Region Year                                                  
Haiti  1990                  187642                  94025   
       1991                  192031                  96229   
       1992                  196578                  98512   
       1993                  201290                 100878   
       1994                  206039                 103264   
       1995                  210681                 105598   
       1996                  214964                 107755   
       1997                  218572                 109577   
       1998                  221386                 111005   
       1999                  223495                 112085   
       2000                  225044                 112886   

             Female Population Age 6  Total Population Age 6  \
Region Year                                                    
Haiti  1990                    93617                  187642   
       1991                    95802                  192031   
       1992                    98066                  196578   
       1993                   100412                  201290   
       1994                   102775                  206039   
       1995                   105083                  210681   
       1996                   107209                  214964   
       1997                   108995                  218572   
       1998                   110381                  221386   
       1999                   111410                  223495   
       2000                   112158                  225044   

             Male Population Age 7  Female Population Age 7  \
Region Year                                                   
Haiti  1990                  94025                    93617   
       1991                  96229                    95802   
       1992                  98512                    98066   
       1993                 100878                   100412   
       1994                 103264                   102775   
       1995                 105598                   105083   
       1996                 107755                   107209   
       1997                 109577                   108995   
       1998                 111005                   110381   
       1999                 112085                   111410   
       2000                 112886                   112158   

             Total Population Age 7  Male Population Age 8  ...  \
Region Year                                                 ...   
Haiti  1990                  187642                  94025  ...   
       1991                  192031                  96229  ...   
       1992                  196578                  98512  ...   
       1993                  201290                 100878  ...   
       1994                  206039                 103264  ...   
       1995                  210681                 105598  ...   
       1996                  214964                 107755  ...   
       1997                  218572                 109577  ...   
       1998                  221386                 111005  ...